In [ ]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic
from typing import Dict, List, Tuple, Any
import time
import json
import os
from tqdm import tqdm
import heapq
import random

In [ ]:
SIZE = [10, 20, 50, 100, 200, 500, 1000]
DENSITY = [0.2, 0.5, 0.8, 1.0]
NOISE_LEVEL = [0.0, 0.1, 0.5, 0.8]
NEGATIVE_VALUES = [False, True]

## Code to create the problem instance

In [ ]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round(), map

## Bellman Ford

In [ ]:
def run_bellman_ford_apsp_and_save(
    problem_matrix: np.ndarray, 
    pos_map: np.ndarray, 
    instance_params: Dict[str, Any], 
    output_filename: str = "bellman_ford_results.jsonl",
):
    """
    Runs All-Pairs Shortest Path using NetworkX's Bellman-Ford (ground truth)
    and saves the results, including cycle detection.
    """
    
    #Setup Graph
    masked = np.ma.masked_array(problem_matrix, mask=np.isinf(problem_matrix))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
    nodes = list(G.nodes())
    N = len(nodes)
    
    apsp_paths = {}
    total_time = 0.0
    total_negative_cycles = 0
    
    #Run Bellman-Ford APSP and Time Measurement
    
    start_time = time.time()
    
    #Run Bellman-Ford for every pair (s, d) and (d, s)
    for s, d in combinations(nodes, 2):
        
        #Check s -> d
        try:
            path_sd = nx.bellman_ford_path(G, s, d, weight='weight')
            cost_sd = nx.path_weight(G, path_sd, weight='weight')
            cycle_sd = False
        except nx.NetworkXUnbounded:
            #Negative cycle detected
            path_sd = None
            cost_sd = -float('inf')
            cycle_sd = True
            total_negative_cycles += 1
        except nx.NetworkXNoPath:
            #Nodes are not connected
            path_sd = None
            cost_sd = float('inf')
            cycle_sd = False
            
        apsp_paths[f"{s}->{d}"] = {"path": path_sd, "cost": cost_sd, "cycle": cycle_sd}
        
        #Check d -> s
        try:
            path_ds = nx.bellman_ford_path(G, d, s, weight='weight')
            cost_ds = nx.path_weight(G, path_ds, weight='weight')
            cycle_ds = False
        except nx.NetworkXUnbounded:
            #Negative cycle detected
            path_ds = None
            cost_ds = -float('inf')
            cycle_ds = True
            total_negative_cycles += 1
        except nx.NetworkXNoPath:
            #Nodes are not connected
            path_ds = None
            cost_ds = float('inf')
            cycle_ds = False
            
        apsp_paths[f"{d}->{s}"] = {"path": path_ds, "cost": cost_ds, "cycle": cycle_ds}
        
    end_time = time.time()
    total_time = end_time - start_time

    #Data Structuring and Saving
    
    experiment_data = {
        "parameters": instance_params,
        "metrics": {
            "total_time_seconds": total_time,
            "number_of_pairs": N * (N - 1),
            #total_negative_cycles counts the number of times the exception was raised
            "total_negative_cycles_raised": total_negative_cycles 
        },
        "problem_matrix": problem_matrix.tolist(), 
        "node_coordinates": pos_map.tolist(), 
        "bellman_ford_paths": apsp_paths,
    }

    #Append to the dedicated Bellman-Ford file
    with open(output_filename, 'a') as f:
        #Use custom default handler for JSON serialization of infinity
        json.dump(experiment_data, f, default=lambda x: str(x) if isinstance(x, float) and abs(x) == float('inf') else x)
        f.write('\n')

    print(f"Successfully ran Bellman-Ford ground truth for size {N} to {output_filename}")

In [ ]:
EXPERIMENT_RUNS = []

#Use itertools.product to get every combination of the four lists
#The product will yield a tuple like (size, density, noise_level, negative_values)
for size, density, noise_level, negative_values in product(
    SIZE, DENSITY, NOISE_LEVEL, NEGATIVE_VALUES
):
    # Construct the dictionary for this specific run
    run_params = {
        'size': size,
        'density': density,
        'noise_level': noise_level,
        'negative_values': negative_values,
    }
    EXPERIMENT_RUNS.append(run_params)

print(f"Total number of experiment runs generated: {len(EXPERIMENT_RUNS)}")
#Print the first few runs to verify the structure
print("Example of the first 3 runs:")
for i in range(3):
    print(EXPERIMENT_RUNS[i])

# Resilience Check: Load Completed Runs
def load_completed_keys(filename: str) -> set:
    """Reads the JSONL file and returns a set of completed parameter tuples."""
    if not os.path.exists(filename):
        return set()
    
    completed = set()
    with open(filename, 'r') as f:
        for line in f:
            try:
                data = json.loads(line)
                p = data['parameters']
                #Create a hashable key for parameter matching
                key = (p['size'], p['density'], p['noise_level'], p['negative_values'])
                completed.add(key)
            except json.JSONDecodeError:
                #Skip corrupted lines
                continue
    return completed

## Run code

In [ ]:
OUTPUT_FILE = "bellman_ford_experiment_results.jsonl"
TOTAL_RUNS = len(EXPERIMENT_RUNS)

completed_keys = load_completed_keys(OUTPUT_FILE)

RESUMABLE_RUNS = []
for params in EXPERIMENT_RUNS:
    key = (params['size'], params['density'], params['noise_level'], params['negative_values'])
    if key not in completed_keys:
        RESUMABLE_RUNS.append(params)

print(f"Total defined runs: {TOTAL_RUNS}")
print(f"Completed runs found: {len(completed_keys)}")
print(f"Remaining runs to execute: {len(RESUMABLE_RUNS)}")
print("-" * 50)

pbar = tqdm(RESUMABLE_RUNS, desc="Running APSP Experiments")

for params in pbar:
    try:
        #Generate the problem instance
        matrix, coordinates = create_problem(**params)
        
        #Run the A* search and save the data
        run_bellman_ford_apsp_and_save(
            problem_matrix=matrix,
            pos_map=coordinates,
            instance_params=params,
            output_filename=OUTPUT_FILE
        )
        
        #Update the progress bar status with current run details
        pbar.set_postfix_str(f"Size: {params['size']}, Neg: {params['negative_values']}")

    except Exception as e:
        #Crucial for resilience: Print error, but continue to the next run
        print(f"\n‼️ ERROR processing run: {params}. Error: {e}")
        continue

print("\n\n🚀 All scheduled experiments complete!")